In [ ]:
!pip install --upgrade google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client markdown2

In [15]:
import os
import markdown2
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Define the markdown meeting notes
markdown_notes = """
# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

# Function to authenticate and create a Google Docs service using a service account
def create_google_docs_service():
    try:
        # Load credentials from the uploaded service account JSON file
        creds = service_account.Credentials.from_service_account_file(
            '/content/sample_data/credentials.json',  # Update this path
            scopes=['https://www.googleapis.com/auth/documents']
        )
        service = build('docs', 'v1', credentials=creds)
        return service
    except Exception as e:
        print(f"Error during authentication: {e}")
        return None

# Function to create a new Google Doc
def create_google_doc(service, title):
    doc = {
        'title': title
    }
    doc = service.documents().create(body=doc).execute()
    return doc.get('documentId')

# Function to convert markdown to Google Docs format
def convert_markdown_to_google_doc(service, document_id, markdown_text):
    # Convert markdown to HTML
    html = markdown2.markdown(markdown_text)

    # Prepare requests for Google Docs API
    requests = []
    current_index = 1  # Start inserting at index 1

    # Split the HTML into lines for processing
    lines = html.splitlines()
    for line in lines:
        current_index = 1
        if line.startswith('<h1>'):
            requests.append({
                'insertText': {
                    'location': {'index': current_index},
                    'text': line.replace('<h1>', '').replace('</h1>', '') + '\n'
                }
            })

        elif line.startswith('<h2>'):
            requests.append({
                'insertText': {
                    'location': {'index': current_index},
                    'text': line.replace('<h2>', '').replace('</h2>', '') + '\n'
                }
            })

        elif line.startswith('<h3>'):
            requests.append({
                'insertText': {
                    'location': {'index': current_index},
                    'text': line.replace('<h3>', '').replace('</h3>', '') + '\n'
                }
            })

        elif line.startswith('<ul>'):
            continue  # Skip the <ul> tag
        elif line.startswith('<li>'):
            requests.append({
                'insertText': {
                    'location': {'index': current_index},
                    'text': line.replace('<li>', '').replace('</li>', '') + '\n'
                }
            })

        elif line.startswith('<p>'):
            requests.append({
                'insertText': {
                    'location': {'index': current_index},
                    'text': line.replace('<p>', '').replace('</p>', '') + '\n'
                }
            })

    # Execute the batch update
    service.documents().batchUpdate(documentId=document_id, body={'requests': requests}).execute()

# Main function to run the script
def main():
    service = create_google_docs_service()
    if service:
        document_id = create_google_doc(service, "Product Team Sync - May 15, 2023")
        convert_markdown_to_google_doc(service, document_id, markdown_notes)
        print(f"Document created with ID: {document_id}")

# Run the main function
if __name__ == "__main__":
    main()

Document created with ID: 1xvD-zVSnzORSStS1akJAj2yxJ5LuCrhNxxoTOXdspKw
